<a href="https://colab.research.google.com/github/arastaravan/turing-apex/blob/main/turing_apex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## initial setup

In [ ]:
## TODO
# add docstring to all functions
# better RAG fetch
# better prompt engineering

In [ ]:
# I use this public folder for storing find tuned models.
# You can import it in order to avoid fine tuning cost on the model.
# "https://drive.google.com/drive/folders/1rIBXkHmwvPdpOvYMcWIPTc7jdyjiHZI4?usp=sharing"

In [ ]:
# load dataset
!gdown "https://drive.google.com/uc?id=1tIkdQtV00yz1kYWtZ0z3tK-PAlnxnGmC" -O data.zip
!unzip data.zip

Downloading...
From: https://drive.google.com/uc?id=1tIkdQtV00yz1kYWtZ0z3tK-PAlnxnGmC
To: /content/data.zip
100% 36.6k/36.6k [00:00<00:00, 74.9MB/s]
Archive:  data.zip
   creating: data/
  inflating: data/train_data.csv     
  inflating: data/test_data.csv      


In [ ]:
import pandas as pd

train_df = pd.read_csv("./data/train_data.csv")
train_df.head()

,prompt,A,B,C,D,E,answer
0,What is the main sequence in astronomy?,The main sequence is a type of galaxy that con...,The main sequence is a type of black hole that...,The main sequence is a continuous and distinct...,The main sequence is a group of planets that o...,The main sequence is a type of nebula that is ...,C
1,"What is the ""ultraviolet catastrophe""?",It is a phenomenon that occurs only in multi-m...,It is the misbehavior of a formula for higher ...,It is the standing wave of a string in harmoni...,It is a flaw in classical physics that results...,It is a disproven theory about the distributio...,B
2,What is the Carnot engine?,The Carnot engine is a theoretical engine that...,The Carnot engine is an ideal heat engine that...,The Carnot engine is a real heat engine that o...,The Carnot engine is a theoretical engine that...,The Carnot engine is a real engine that operat...,B
3,What is accelerator-based light-ion fusion?,Accelerator-based light-ion fusion is a techni...,Accelerator-based light-ion fusion is a techni...,Accelerator-based light-ion fusion is a techni...,Accelerator-based light-ion fusion is a techni...,Accelerator-based light-ion fusion is a techni...,A
4,"What is a ""coffee ring"" in physics?",A type of coffee that is made by boiling coffe...,A pattern left by a particle-laden liquid afte...,A type of coffee that is made by mixing instan...,A type of coffee that is made by pouring hot w...,A pattern left by a particle-laden liquid afte...,E


In [ ]:
test_df = pd.read_csv("./data/test_data.csv")
test_df.head()

,prompt,A,B,C,D,E
0,What is the Droste effect?,The Droste effect is a type of optical illusio...,The Droste effect is a type of packaging desig...,The Droste effect is a type of painting techni...,The Droste effect is a recursive image effect ...,The Droste effect is a type of recursive algor...
1,What is the purpose of measuring the Larmor pr...,To measure the magnetization in the same direc...,To create a T1-weighted image that is useful f...,To obtain sufficient signal quality in the mic...,To measure the independent relaxation processe...,To change the repetition time (TR) and obtain ...
2,Which of the following statements accurately d...,Gauss's law holds only for situations involvin...,"Gauss's law holds in all cases, but it is most...","Gauss's law, which applies equally to all elec...",Gauss's law only holds for electric fields wit...,"Gauss's law, which holds for all situations, i..."
3,What is the significance of the discovery of t...,The discovery of the Crab pulsar confirmed the...,The discovery of the Crab pulsar confirmed the...,The discovery of the Crab pulsar confirmed the...,The discovery of the Crab pulsar disproved the...,The discovery of the Crab pulsar confirmed the...
4,What is the Einstein@Home project?,The Einstein@Home project is a project that ai...,The Einstein@Home project is a project that ai...,The Einstein@Home project is a distributed com...,The Einstein@Home project is a project that ai...,The Einstein@Home project is a project that ai...


In [ ]:
# install relevant python libraries
!pip install wikipedia datasets accelerate peft bitsandbytes -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:

## model setup

In [ ]:
#### this cell contains LLM model setup.
#### anyone changing the model should take care not changing the interface provided by class Model to the rest of the code

import os
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


class Model:
    # def format_prompts(queries):
    #     prompts = []
    #     for c,p in zip(queries["contexts"], queries["prompts"]):
    #         prompts.append(f"Context: {c}\nInstruct: {p}\nOutput:")
    #     return prompts

    def format_prompt(query, few_shots=None):
        prompt = f"Context: {query['context']}\nInstruct: {query['prompt']}\nOutput:"
        if few_shots:
            prompt = few_shots + "\n" + prompt
        return prompt

    def __init__(self):
        ## load model
        # load tokenizer and model
        self.model_id = "microsoft/phi-2"
        self._tokenizer = AutoTokenizer.from_pretrained(self.model_id)
        self._tokenizer.pad_token = self._tokenizer.eos_token
        self._tokenizer.padding_side = "left"

        # 8-bit quantization config
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_quant_type="nf4",
            bnb_8bit_compute_dtype=torch.float16
        )

        # load model
        self._model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16,
            device_map="auto",
            quantization_config=bnb_config
        )

        # set model to evaluation mode
        self._model.eval()

    def fine_tune(self, dataset_id, format_example, save_path, max_new_data=100): # not used in this notebook
        if os.path.exists(save_path):
            print("save_path argument already holds a model. exiting.")
            return

        dataset = load_dataset(dataset_id, split="train")
        # format dataset into prompts
        dataset = dataset.map(format_example)
        dataset = dataset.select(range(min(max_new_data,len(dataset))))

        # Tokenize dataset
        def tokenize_function(example):
            return self._tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

        tokenized_dataset = dataset.map(tokenize_function, batched=True)

        # Set format for PyTorch
        tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

        # Prepare for LoRA training
        model = prepare_model_for_kbit_training(self._model)

        # Configure LoRA
        lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["q_proj", "v_proj"],  # you can customize this per model architecture
            lora_dropout=0.1,
            bias="none",
            task_type="CAUSAL_LM"
        )

        model = get_peft_model(model, lora_config)

        # Training arguments
        training_args = TrainingArguments(
            output_dir=save_path,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            num_train_epochs=3,
            logging_dir=f"{save_path}/logs",
            logging_steps=10,
            fp16=True,
            save_total_limit=1,
            save_steps=50,
            remove_unused_columns=False,
            report_to="none"
        )

        # Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset,
            tokenizer=self._tokenizer,
            data_collator=DataCollatorForLanguageModeling(self._tokenizer, mlm=False)
        )

        # Train
        trainer.train()

        # Save final LoRA adapter
        model.save_pretrained(save_path)
        self._tokenizer.save_pretrained(save_path)

        print(f"LoRA-finetuned model saved to: {save_path}")

    def load_local_model(self, load_path):
        if not os.path.exists(load_path):
            print("specified path does not exist on content/drive/MyDrive")
            return

        # load tokenizer
        self._tokenizer = AutoTokenizer.from_pretrained(load_path)
        self._tokenizer.pad_token = self._tokenizer.eos_token
        self._tokenizer.padding_side = "left"

        # 8-bit quantization config
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_quant_type="nf4",
            bnb_8bit_compute_dtype=torch.float16
        )

        # load model
        self._model = AutoModelForCausalLM.from_pretrained(
            load_path,
            torch_dtype=torch.float16,
            device_map="auto",
            quantization_config=bnb_config
        )

        self._model.eval()

    def save_model(self, save_path): #TODO: handle errors here
        os.makedirs(save_path, exist_ok=True)
        self._model.save_pretrained(save_path)
        self._tokenizer.save_pretrained(save_path)

    def predict(self, prompt, max_new_tokens=200):
        # prompt = Model.format_prompt(query)
        inputs = model._tokenizer([prompt], return_tensors="pt").to(model._model.device)
        with torch.no_grad():
            outputs = model._model.generate(**inputs, max_new_tokens=max_new_tokens)
        response = model._tokenizer.batch_decode(outputs)[0]
        return response[len(prompt):].strip()

    def batch_predict(self, prompts):
        responses = []

        for i in range(0, len(prompts), 5):
            for prompt in prompts[i:min(i+5,len(prompts))]:
                resp = self.predict(prompt)
                responses.append(resp)
            torch.cuda.empty_cache()

        return responses

    # def batch_response(self, queries, max_new_tokens=200):
    #     prompts = Model.format_prompts(queries)
    #     inputs = self._tokenizer(prompts, return_tensors="pt").to(self._model.device)
    #     with torch.no_grad():
    #         outputs = self._model.generate(**inputs, max_new_tokens=max_new_tokens)
    #     responses = self._tokenizer.batch_decode(outputs)

    #     # remove prompt text from model response text for each response
    #     responses = [(r[len(p):]).strip() for p, r in zip(prompts, responses)]

        return responses

model = Model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# prompts = ["""Context: In astronomy, the main sequence is a classification of stars which appear on plots of stellar color versus brightness as a continuous and distinctive band. Stars on this band are known as main-sequence stars or dwarf stars, and positions of stars on and off the band are believed to indicate their physical properties, as well as their progress through several types of star life-cycles. These are the most numerous true stars in the universe and include the Sun. Color-magnitude plots are known as Hertzsprung–Russell diagrams after Ejnar Hertzsprung and Henry Norris Russell. After condensation and ignition of a star, it generates thermal energy in its dense core region through nuclear fusion of hydrogen into helium. During this stage of the star's lifetime, it is located on the main sequence at a position determined primarily by its mass but also based on its chemical composition and age. The cores of main-sequence stars are in hydrostatic equilibrium, where outward thermal pressure from the hot core is balanced by the inward pressure of gravitational collapse from the overlying layers. The strong dependence of the rate of energy generation on temperature and pressure helps to sustain this balance. Energy generated at the core makes its way to the surface and is radiated away at the photosphere. The energy is carried by either radiation or convection, with the latter occurring in regions with steeper temperature gradients, higher opacity, or both. The main sequence is sometimes divided into upper and lower parts, based on the dominant process that a star uses to generate energy. The Sun, along with main sequence stars below about 1.5 times the mass of the Sun (1.5 M☉), primarily fuse hydrogen atoms together in a series of stages to form helium, a sequence called the proton–proton chain. Above this mass, in the upper main sequence, the nuclear fusion process mainly uses atoms of carbon, nitrogen, and oxygen as intermediaries in the CNO cycle that produces helium from hydrogen atoms. Main-sequence stars with more than two solar masses undergo convection in their core regions, which acts to stir up the newly created helium and maintain the proportion of fuel needed for fusion to occur. Below this mass, stars have cores that are entirely radiative with convective zones near the surface. With decreasing stellar mass, the proportion of the star forming a convective envelope steadily increases. The main-sequence stars below 0.4 M☉ undergo convection throughout their mass. When core convection does not occur, a helium-rich core develops surrounded by an outer layer of hydrogen. The more massive a star is, the shorter its lifespan on the main sequence. After the hydrogen fuel at the core has been consumed, the star evolves away from the main sequence on the HR diagram, into a supergiant, red giant, or directly to a white dwarf.
# Instruct: What is the main sequence in astronomy?  \nA- The main sequence is a type of black hole that is formed from the collapse of a massive star.\nB- The main sequence is a type of galaxy that contains a large number of stars.\nC- The main sequence is a continuous and distinctive band of stars that appears on plots of stellar color versus brightness. Stars on this band are known as main-sequence stars or dwarf stars.\nD- The main sequence is a group of planets that orbit around a star in a solar system.\nE- The main sequence is a type of nebula that is formed from the explosion of a supernova. \n
# Output:"""]

RAG setup

In [ ]:
import wikipedia

def fetch_wikipedia_passages(query, n=3):
    pages = wikipedia.search(query)
    texts = []
    for title in pages[:n]:
        try:
            summary = wikipedia.summary(title)
            texts.append(summary)
        except:
            continue
    if (len(texts) == 0):
        return [""]
    return texts


In [ ]:
def _format_prompt(row, answer=None):
    return (
        f"{row['prompt']}\n"
        f"A. {row['A']}\n"
        f"B. {row['B']}\n"
        f"C. {row['C']}\n"
        f"D. {row['D']}\n"
        f"E. {row['E']}"
    )

def _format_context(row):
    context = fetch_wikipedia_passages(row["prompt"])[0]
    return context.replace('\n', ' ')

def format_queries(df):
    '''
    formats a general Q&A format query for model which is independent of the model
    i.e. prompt engineering is **NOT** done here, but in the Model class definition

    -- input format:
    dataframe, having at least headers "prompt" and "A"-"E"

    -- output format:
    a list of dictionaries consisting of:
    "context": provides context
    "prompt": actual question + answers, seperated by '\n'
    '''
    prompts = list(df.apply(_format_prompt, axis=1))
    contexts = list(df.apply(_format_context, axis=1))
    queries = [{"context": c, "prompt": p} for c, p in zip(contexts, prompts)]
    return queries


In [ ]:
queries = format_queries(train_df)
prompts = [Model.format_prompt(q) for q in queries]

/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
responses = model.batch_predict(prompts)

# responses = []
# for i in range(0, 50, 10):
#     for prompt in prompts[i:i+10]:
#         resp = model.predict(prompt)[0]
#         responses.append(resp)
#     torch.cuda.empty_cache() # put this in model later on, using batch_predict

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
# accuracy
def model_accuracy(responses, answers):
    return np.mean(np.array(responses) == np.array(answers))

In [ ]:
answers = train_df["answer"]

model_accuracy([r[0] for r in responses], answers)

np.float64(0.5)

In [ ]:
np.sum(np.array(answers) == 'A')

np.int64(7)

In [ ]:
# def format_example(example):
#     perm = np.random.permutation(["distractor1","distractor2","distractor3","correct_answer"])
#     prompt = f"Context: {example['support']} \nInstruct: {example['question']}\n \nA. {example[perm[0]]}\nB. {example[perm[1]]}\nC. {example[perm[2]]}\nD. {example[perm[3]]}\nOutput:"
#     label = f" {example['correct_answer']}"
#     return {"text": prompt + label}

# model.fine_tune("allenai/sciq", format_example, "/content/drive/MyDrive/colab_models/finetune_allenai_sciq")

In [ ]:
responses

['C',
 'D',
 'T',
 'A',
 'E',
 'E',
 'D',
 'B',
 'E',
 'A',
 'C',
 'A',
 'E',
 'B',
 'C',
 'C',
 'C',
 'A',
 'T',
 'A',
 'C',
 'B',
 'B',
 'E',
 'D',
 'B',
 'A',
 'E',
 'D',
 'T',
 'A',
 'e',
 'B',
 'C',
 'A',
 'C',
 'E',
 'A',
 'B',
 'T',
 'C',
 'A',
 'E',
 'T',
 'T',
 'C',
 'C',
 '.',
 'A',
 'T']

In [ ]:
model.predict(queries[2])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'The correct answer is A. The Carnot engine is a theoretical engine that operates in the limiting mode of extreme speed known as dynamic. It represents the theoretical maximum efficiency of a heat engine operating between any two given thermal or heat reservoirs at different temperatures.\n<|endoftext|>'

In [ ]:
shot_responses = model.batch_predict(prompts[:3])
# for p in prompts[:3]:
#     shot_responses.append(model.predict(p))
print(shot_responses)
print(answers[:3])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['C. The main sequence is a continuous and distinctive band of stars that appears on plots of stellar color versus brightness. Stars on this band are known as main-sequence stars or dwarf stars.\n<|endoftext|>', 'D. It is a flaw in classical physics that results in the misallocation of energy.\nThe correct answer is D. The ultraviolet catastrophe is a flaw in classical physics that results in the misallocation of energy.\nInstruct: What is the correct answer?\nA. The ultraviolet catastrophe is a phenomenon that occurs only in multi-mode vibration.\nB. It is the misbehavior of a formula for higher frequencies.\nC. It is the standing wave of a string in harmonic resonance.\nD. It is a flaw in classical physics that results in the misallocation of energy.\nE. It is a disproven theory about the distribution of electromagnetic radiation.\nThe correct answer is D. The ultraviolet catastrophe is a', 'The correct answer is A. The Carnot engine is a theoretical engine that operates in the limit

In [ ]:
few_shots = '\n'.join([f"{p} {answers[i]}\n<|endoftext|>" for i, p in enumerate(prompts[:3])])

In [ ]:
few_shots

'Context: In astronomy, the main sequence is a classification of stars which appear on plots of stellar color versus brightness as a continuous and distinctive band. Stars on this band are known as main-sequence stars or dwarf stars, and positions of stars on and off the band are believed to indicate their physical properties, as well as their progress through several types of star life-cycles. These are the most numerous true stars in the universe and include the Sun. Color-magnitude plots are known as Hertzsprung–Russell diagrams after Ejnar Hertzsprung and Henry Norris Russell.  After condensation and ignition of a star, it generates thermal energy in its dense core region through nuclear fusion of hydrogen into helium. During this stage of the star\'s lifetime, it is located on the main sequence at a position determined primarily by its mass but also based on its chemical composition and age. The cores of main-sequence stars are in hydrostatic equilibrium, where outward thermal pre

In [ ]:
few_shot_prompts = [Model.format_prompt(q, few_shots) for q in queries[3:]]

In [ ]:
few_shot_prompts[0]

'Context: In astronomy, the main sequence is a classification of stars which appear on plots of stellar color versus brightness as a continuous and distinctive band. Stars on this band are known as main-sequence stars or dwarf stars, and positions of stars on and off the band are believed to indicate their physical properties, as well as their progress through several types of star life-cycles. These are the most numerous true stars in the universe and include the Sun. Color-magnitude plots are known as Hertzsprung–Russell diagrams after Ejnar Hertzsprung and Henry Norris Russell.  After condensation and ignition of a star, it generates thermal energy in its dense core region through nuclear fusion of hydrogen into helium. During this stage of the star\'s lifetime, it is located on the main sequence at a position determined primarily by its mass but also based on its chemical composition and age. The cores of main-sequence stars are in hydrostatic equilibrium, where outward thermal pre

In [ ]:
few_shot_responses = model.batch_predict(few_shot_prompts)

# few_shot_responses = []

# for i in range(0, len(few_shot_prompts), 5):
#     for prompt in few_shot_prompts[i:min(i+5,len(few_shot_prompts))]:
#         resp = model.predict(prompt)
#         few_shot_responses.append(resp)
#     torch.cuda.empty_cache() # put this in model later on, using batch_predict

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [ ]:
model_accuracy([f[0] for f in few_shot_responses], answers[3:])

np.float64(0.5957446808510638)

In [ ]:
[f[0] for f in few_shot_responses]

['A',
 'B',
 'E',
 'D',
 'B',
 'E',
 'A',
 'C',
 'A',
 'A',
 'C',
 'A',
 'C',
 'C',
 'A',
 'B',
 'A',
 'C',
 'E',
 'B',
 'E',
 'C',
 'B',
 'A',
 'E',
 'D',
 'T',
 'A',
 'e',
 'B',
 'B',
 '’',
 'A',
 'E',
 'A',
 'B',
 'D',
 'C',
 'C',
 'E',
 'E',
 'B',
 'C',
 'C',
 '.',
 'A',
 'B']

In [ ]:
test_df.head()

,prompt,A,B,C,D,E
0,What is the Droste effect?,The Droste effect is a type of optical illusio...,The Droste effect is a type of packaging desig...,The Droste effect is a type of painting techni...,The Droste effect is a recursive image effect ...,The Droste effect is a type of recursive algor...
1,What is the purpose of measuring the Larmor pr...,To measure the magnetization in the same direc...,To create a T1-weighted image that is useful f...,To obtain sufficient signal quality in the mic...,To measure the independent relaxation processe...,To change the repetition time (TR) and obtain ...
2,Which of the following statements accurately d...,Gauss's law holds only for situations involvin...,"Gauss's law holds in all cases, but it is most...","Gauss's law, which applies equally to all elec...",Gauss's law only holds for electric fields wit...,"Gauss's law, which holds for all situations, i..."
3,What is the significance of the discovery of t...,The discovery of the Crab pulsar confirmed the...,The discovery of the Crab pulsar confirmed the...,The discovery of the Crab pulsar confirmed the...,The discovery of the Crab pulsar disproved the...,The discovery of the Crab pulsar confirmed the...
4,What is the Einstein@Home project?,The Einstein@Home project is a project that ai...,The Einstein@Home project is a project that ai...,The Einstein@Home project is a distributed com...,The Einstein@Home project is a project that ai...,The Einstein@Home project is a project that ai...
